# Defining a Fixed-Floating Interest Rate Swap

Simple introduction to the inputs required to define a Ibor deposit and a FRA

In [1]:
from financepy.utils import *
from financepy.products.rates import *

####################################################################
# FINANCEPY BETA Version 0.200 - This build:  14 Jul 2021 at 13:18 #
# **** NEW PEP8 COMPLIANT VERSION -- PLEASE UPDATE YOUR CODE  **** #
#      This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy  #
#      Send any bug reports or comments to quant@financepy.com     #
####################################################################



## Creating the Swap

We begin by setting the market conventions we wish to use.

In [2]:
swapCalendarType = CalendarTypes.TARGET
bus_day_adjust_type = BusDayAdjustTypes.FOLLOWING
date_gen_rule_type = DateGenRuleTypes.BACKWARD

We then define the Fixed Leg

In [3]:
fixed_coupon = 0.05
fixedFreqType = FrequencyTypes.ANNUAL
fixed_day_count_type = DayCountTypes.ACT_360

And then we define the Floating Leg

In [4]:
float_spread = 0.0
floatFreqType = FrequencyTypes.SEMI_ANNUAL
float_day_count_type = DayCountTypes.ACT_360

We need to specify the size and direction of the swap.

In [5]:
swapType = SwapTypes.RECEIVE
notional = 10 * ONE_MILLION

And then we define when the swap begins and ends.

In [6]:
start_date = Date(20, 6, 2018)
maturity_date = Date(20, 9, 2025)

We can now create the swap object.

In [7]:
swap = IborSwap(start_date,
                    maturity_date,
                    swapType,
                    fixed_coupon,
                    fixedFreqType,
                    fixed_day_count_type,
                    notional,
                    float_spread,
                    floatFreqType,
                    float_day_count_type,
                    swapCalendarType,
                    bus_day_adjust_type,
                    date_gen_rule_type)

In [8]:
print(swap)

OBJECT TYPE: IborSwap
OBJECT TYPE: SwapFixedLeg
START DATE: 20-JUN-2018
TERMINATION DATE: 20-SEP-2025
MATURITY DATE: 22-SEP-2025
NOTIONAL: 10000000
PRINCIPAL: 0.0
LEG TYPE: SwapTypes.RECEIVE
COUPON: 0.05
FREQUENCY: FrequencyTypes.ANNUAL
DAY COUNT: DayCountTypes.ACT_360
CALENDAR: CalendarTypes.TARGET
BUS DAY ADJUST: BusDayAdjustTypes.FOLLOWING
DATE GEN TYPE: DateGenRuleTypes.BACKWARD

OBJECT TYPE: SwapFloatLeg
START DATE: 20-JUN-2018
TERMINATION DATE: 20-SEP-2025
MATURITY DATE: 22-SEP-2025
NOTIONAL: 10000000
SWAP TYPE: SwapTypes.PAY
SPREAD (BPS): 0.0
FREQUENCY: FrequencyTypes.SEMI_ANNUAL
DAY COUNT: DayCountTypes.ACT_360
CALENDAR: CalendarTypes.TARGET
BUS DAY ADJUST: BusDayAdjustTypes.FOLLOWING
DATE GEN TYPE: DateGenRuleTypes.BACKWARD



In [9]:
swap.print_flows()

START DATE: 20-JUN-2018
MATURITY DATE: 22-SEP-2025
COUPON (%): 5.0
FREQUENCY: FrequencyTypes.ANNUAL
DAY COUNT: DayCountTypes.ACT_360
PAY_DATE     ACCR_START   ACCR_END      DAYS  YEARFRAC    RATE      PAYMENT 
20-SEP-2018  20-JUN-2018  20-SEP-2018    92  0.255556  5.000000    127777.78
20-SEP-2019  20-SEP-2018  20-SEP-2019   365  1.013889  5.000000    506944.44
21-SEP-2020  20-SEP-2019  21-SEP-2020   367  1.019444  5.000000    509722.22
20-SEP-2021  21-SEP-2020  20-SEP-2021   364  1.011111  5.000000    505555.56
20-SEP-2022  20-SEP-2021  20-SEP-2022   365  1.013889  5.000000    506944.44
20-SEP-2023  20-SEP-2022  20-SEP-2023   365  1.013889  5.000000    506944.44
20-SEP-2024  20-SEP-2023  20-SEP-2024   366  1.016667  5.000000    508333.33
22-SEP-2025  20-SEP-2024  22-SEP-2025   367  1.019444  5.000000    509722.22
START DATE: 20-JUN-2018
MATURITY DATE: 22-SEP-2025
SPREAD (bp): 0.0
FREQUENCY: FrequencyTypes.SEMI_ANNUAL
DAY COUNT: DayCountTypes.ACT_360
PAY_DATE     ACCR_START   ACCR_END 

## Valuation

In [10]:
valuation_date = Date(20,3,2020)

In [11]:
settlement_date = valuation_date.add_weekdays(0)

This is just using a flat curve without any Ibor accrual factors.

In [12]:
from financepy.market.curves.discount_curve_flat import DiscountCurveFlat

In [13]:
discount_curve = DiscountCurveFlat(valuation_date, 0.05, FrequencyTypes.SEMI_ANNUAL)

Need to specify the next fixing on the floating leg

In [14]:
swap.value(settlement_date, discount_curve, discount_curve)

254993.13993501337

We can examine the individual legs - here is the floating leg valuation

In [15]:
swap.print_float_leg_pv()

START DATE: 20-JUN-2018
MATURITY DATE: 22-SEP-2025
SPREAD (BPS): 0.0
FREQUENCY: FrequencyTypes.SEMI_ANNUAL
DAY COUNT: DayCountTypes.ACT_360
PAY_DATE     ACCR_START   ACCR_END     DAYS  YEARFRAC    IBOR      PAYMENT       DF          PV        CUM PV
24-SEP-2018  20-JUN-2018  24-SEP-2018    96  0.266667    0.00000         0.00  0.00000000         0.00         0.00
22-MAR-2019  24-SEP-2018  22-MAR-2019   179  0.497222    0.00000         0.00  0.00000000         0.00         0.00
23-SEP-2019  22-MAR-2019  23-SEP-2019   185  0.513889    0.00000         0.00  0.00000000         0.00         0.00
23-MAR-2020  23-SEP-2019  23-MAR-2020   182  0.505556    0.08009      4048.79  0.99959528      4047.15      4047.15
22-SEP-2020  23-MAR-2020  22-SEP-2020   183  0.508333    4.91803    250000.00  0.97521491    243803.73    247850.88
22-MAR-2021  22-SEP-2020  22-MAR-2021   181  0.502778    4.92339    247537.32  0.95165783    235570.83    483421.71
22-SEP-2021  22-MAR-2021  22-SEP-2021   184  0.511111 

And here is the fixed leg valuation

In [16]:
swap.print_fixed_leg_pv()

START DATE: 20-JUN-2018
MATURITY DATE: 22-SEP-2025
COUPON (%): 5.0
FREQUENCY: FrequencyTypes.ANNUAL
DAY COUNT: DayCountTypes.ACT_360
PAY_DATE     ACCR_START   ACCR_END     DAYS  YEARFRAC    RATE      PAYMENT       DF          PV        CUM PV
20-SEP-2018  20-JUN-2018  20-SEP-2018    92  0.255556   5.00000    127777.78  0.00000000         0.00         0.00
20-SEP-2019  20-SEP-2018  20-SEP-2019   365  1.013889   5.00000    506944.44  0.00000000         0.00         0.00
21-SEP-2020  20-SEP-2019  21-SEP-2020   367  1.019444   5.00000    509722.22  0.97534651    497155.79    497155.79
20-SEP-2021  21-SEP-2020  20-SEP-2021   364  1.011111   5.00000    505555.56  0.92850948    469413.12    966568.91
20-SEP-2022  20-SEP-2021  20-SEP-2022   365  1.013889   5.00000    506944.44  0.88376869    448021.63   1414590.54
20-SEP-2023  20-SEP-2022  20-SEP-2023   365  1.013889   5.00000    506944.44  0.84118376    426433.43   1841023.97
20-SEP-2024  20-SEP-2023  20-SEP-2024   366  1.016667   5.00000    

In [17]:
swap.pv01(valuation_date, discount_curve)

5.272690754192439

In [18]:
swap.swap_rate(valuation_date, discount_curve)

0.04516388971357023

## Including a final exchange of principal

In [19]:
swap.value(settlement_date, discount_curve, discount_curve)

254993.13993501337

In [20]:
swap.print_float_leg_pv()

START DATE: 20-JUN-2018
MATURITY DATE: 22-SEP-2025
SPREAD (BPS): 0.0
FREQUENCY: FrequencyTypes.SEMI_ANNUAL
DAY COUNT: DayCountTypes.ACT_360
PAY_DATE     ACCR_START   ACCR_END     DAYS  YEARFRAC    IBOR      PAYMENT       DF          PV        CUM PV
24-SEP-2018  20-JUN-2018  24-SEP-2018    96  0.266667    0.00000         0.00  0.00000000         0.00         0.00
22-MAR-2019  24-SEP-2018  22-MAR-2019   179  0.497222    0.00000         0.00  0.00000000         0.00         0.00
23-SEP-2019  22-MAR-2019  23-SEP-2019   185  0.513889    0.00000         0.00  0.00000000         0.00         0.00
23-MAR-2020  23-SEP-2019  23-MAR-2020   182  0.505556    0.08009      4048.79  0.99959528      4047.15      4047.15
22-SEP-2020  23-MAR-2020  22-SEP-2020   183  0.508333    4.91803    250000.00  0.97521491    243803.73    247850.88
22-MAR-2021  22-SEP-2020  22-MAR-2021   181  0.502778    4.92339    247537.32  0.95165783    235570.83    483421.71
22-SEP-2021  22-MAR-2021  22-SEP-2021   184  0.511111 

As expected we see the final PV of the floating Leg is par (same index and discounting curve).

In [21]:
swap.print_fixed_leg_pv()

START DATE: 20-JUN-2018
MATURITY DATE: 22-SEP-2025
COUPON (%): 5.0
FREQUENCY: FrequencyTypes.ANNUAL
DAY COUNT: DayCountTypes.ACT_360
PAY_DATE     ACCR_START   ACCR_END     DAYS  YEARFRAC    RATE      PAYMENT       DF          PV        CUM PV
20-SEP-2018  20-JUN-2018  20-SEP-2018    92  0.255556   5.00000    127777.78  0.00000000         0.00         0.00
20-SEP-2019  20-SEP-2018  20-SEP-2019   365  1.013889   5.00000    506944.44  0.00000000         0.00         0.00
21-SEP-2020  20-SEP-2019  21-SEP-2020   367  1.019444   5.00000    509722.22  0.97534651    497155.79    497155.79
20-SEP-2021  21-SEP-2020  20-SEP-2021   364  1.011111   5.00000    505555.56  0.92850948    469413.12    966568.91
20-SEP-2022  20-SEP-2021  20-SEP-2022   365  1.013889   5.00000    506944.44  0.88376869    448021.63   1414590.54
20-SEP-2023  20-SEP-2022  20-SEP-2023   365  1.013889   5.00000    506944.44  0.84118376    426433.43   1841023.97
20-SEP-2024  20-SEP-2023  20-SEP-2024   366  1.016667   5.00000    

In [22]:
swap._fixed_leg.value(settlement_date, discount_curve)

2636345.37709622

In [23]:
swap._float_leg.value(settlement_date, discount_curve, discount_curve)

-2381352.2371612065

Copyright (c) 2020 Dominic O'Kane